In [1]:
import numpy as np
import tensorflow as tf

### Load Data testing untuk semua skenario

In [2]:
x_test=np.load('./x_test.npy')
x_test

array([[ 216, 1811,  308, ...,   41,   50,  157],
       [ 399,  345,   56, ...,    0,    0,    0],
       [ 467,  360,  116, ...,    0,    0,    0],
       ...,
       [4960,  302,  148, ...,    2,   15,  255],
       [   2,    3,   43, ...,    0,    0,    0],
       [5882,  334,   15, ...,    0,    0,    0]])

In [3]:
y_test=np.load('./y_test.npy')
y_test

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

### Definisikan metrics f1-score

In [4]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

### Mencoba Skenario Sederhana Arsitektur

In [5]:
lstm_model=tf.keras.models.load_model('../Model/Simple/Architecture/lstm.h5',custom_objects={'f1_m':f1_m})
lstm_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 15, 16)            111920    
                                                                 
 lstm_2 (LSTM)               (None, 15, 64)            20736     
                                                                 
 flatten_2 (Flatten)         (None, 960)               0         
                                                                 
 dense_2 (Dense)             (None, 3)                 2883      
                                                                 
Total params: 135,539
Trainable params: 135,539
Non-trainable params: 0
_________________________________________________________________


In [6]:
lstm_model.evaluate(x_test,y_test)

17/17 [==============================] - 5s 5ms/step - loss: 0.9279 - accuracy: 0.7831 - f1_m: 0.7868


[0.9279114603996277, 0.7831094264984131, 0.786816418170929]

In [7]:
bilstm_model=tf.keras.models.load_model('../Model/Simple/Architecture/bilstm.h5',custom_objects={'f1_m':f1_m})
bilstm_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 15, 16)            111920    
                                                                 
 bidirectional_1 (Bidirectio  (None, 15, 128)          41472     
 nal)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 1920)              0         
                                                                 
 dense_3 (Dense)             (None, 3)                 5763      
                                                                 
Total params: 159,155
Trainable params: 159,155
Non-trainable params: 0
_________________________________________________________________


In [8]:
bilstm_model.evaluate(x_test,y_test)

17/17 [==============================] - 1s 8ms/step - loss: 0.7966 - accuracy: 0.8157 - f1_m: 0.8197


[0.7966405749320984, 0.8157389760017395, 0.8197032809257507]

### Mencoba skenario sederhana activation function

In [9]:
import os
BASE_PATH='../Model/Simple/Activation/'

In [10]:
for model in os.listdir(BASE_PATH):
    template=f'Model {model[:-3]}'
    model=tf.keras.models.load_model(os.path.join(BASE_PATH,model),custom_objects={'f1_m':f1_m})
    print('Evaluating',template)
    model.evaluate(x_test,y_test)

Evaluating Model relu_sigmoid
17/17 [==============================] - 1s 14ms/step - loss: 0.5836 - accuracy: 0.8273 - f1_m: 0.8319
Evaluating Model relu_softmax
17/17 [==============================] - 1s 14ms/step - loss: 0.4453 - accuracy: 0.8426 - f1_m: 0.8420
Evaluating Model sigmoid_sigmoid
17/17 [==============================] - 1s 12ms/step - loss: 0.9002 - accuracy: 0.7946 - f1_m: 0.7146
Evaluating Model sigmoid_softmax
17/17 [==============================] - 1s 13ms/step - loss: 0.8918 - accuracy: 0.7524 - f1_m: 0.7598
Evaluating Model tanh_sigmoid
17/17 [==============================] - 1s 7ms/step - loss: 0.5462 - accuracy: 0.8177 - f1_m: 0.7745
Evaluating Model tanh_softmax
17/17 [==============================] - 1s 7ms/step - loss: 0.5089 - accuracy: 0.8157 - f1_m: 0.8182


### Mencoba skenario sederhana optimizer

In [11]:
adam=tf.keras.models.load_model('../Model/Simple/Optimizer/adam.h5',custom_objects={'f1_m':f1_m})
adam.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 16)            111920    
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          41472     
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 3)                 5763      
                                                                 
Total params: 159,155
Trainable params: 159,155
Non-trainable params: 0
_________________________________________________________________


In [12]:
adam.evaluate(x_test,y_test)

17/17 [==============================] - 1s 13ms/step - loss: 0.6350 - accuracy: 0.8157 - f1_m: 0.8204


[0.6350417137145996, 0.8157389760017395, 0.8204213380813599]

In [13]:
rmsprop=tf.keras.models.load_model('../Model/Simple/Optimizer/rmsprop.h5',custom_objects={'f1_m':f1_m})
rmsprop.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 15, 16)            111920    
                                                                 
 bidirectional_1 (Bidirectio  (None, 15, 128)          41472     
 nal)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 1920)              0         
                                                                 
 dense_1 (Dense)             (None, 3)                 5763      
                                                                 
Total params: 159,155
Trainable params: 159,155
Non-trainable params: 0
_________________________________________________________________


In [14]:
rmsprop.evaluate(x_test,y_test)

17/17 [==============================] - 1s 13ms/step - loss: 0.7578 - accuracy: 0.8445 - f1_m: 0.8471


[0.7578050494194031, 0.8445297479629517, 0.8470896482467651]

### Mencoba skenario kompleks arsitektur

In [24]:
lstm_complex=tf.keras.models.load_model('../Model/Complex/Architecture/lstm.h5',custom_objects={'f1_m':f1_m})
lstm_complex.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 15, 128)           895360    
                                                                 
 lstm_12 (LSTM)              (None, 15, 128)           131584    
                                                                 
 lstm_13 (LSTM)              (None, 15, 128)           131584    
                                                                 
 lstm_14 (LSTM)              (None, 15, 128)           131584    
                                                                 
 flatten_4 (Flatten)         (None, 1920)              0         
                                                                 
 dense_8 (Dense)             (None, 512)               983552    
                                                                 
 dropout_4 (Dropout)         (None, 512)              

In [25]:
lstm_complex.evaluate(x_test,y_test)

17/17 [==============================] - 2s 33ms/step - loss: 0.7413 - accuracy: 0.8695 - f1_m: 0.8670


[0.7412736415863037, 0.8694817423820496, 0.8670119643211365]

In [26]:
bilstm_complex=tf.keras.models.load_model('../Model/Complex/Architecture/bilstm.h5',custom_objects={'f1_m':f1_m})
bilstm_complex.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 15, 128)           895360    
                                                                 
 bidirectional_6 (Bidirectio  (None, 15, 256)          263168    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 15, 256)          394240    
 nal)                                                            
                                                                 
 bidirectional_8 (Bidirectio  (None, 15, 256)          394240    
 nal)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 3840)              0         
                                                      

In [27]:
bilstm_complex.evaluate(x_test,y_test)

17/17 [==============================] - 3s 53ms/step - loss: 0.3686 - accuracy: 0.8944 - f1_m: 0.8916


[0.3685898184776306, 0.8944337964057922, 0.8916236758232117]

### Mencoba skenario kompleks optimizer

In [29]:
adam_complex=tf.keras.models.load_model('../Model/Complex/Optimizer/adam.h5',custom_objects={'f1_m':f1_m})
adam_complex.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 15, 128)           895360    
                                                                 
 bidirectional_5 (Bidirectio  (None, 15, 256)          263168    
 nal)                                                            
                                                                 
 bidirectional_6 (Bidirectio  (None, 15, 256)          394240    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 15, 256)          394240    
 nal)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 3840)              0         
                                                      

In [33]:
adam_complex.evaluate(x_test,y_test)

17/17 [==============================] - 2s 56ms/step - loss: 0.6145 - accuracy: 0.8964 - f1_m: 0.8990


[0.614516019821167, 0.8963531851768494, 0.8990137577056885]

In [30]:
rmsprop_complex=tf.keras.models.load_model('../Model/Complex/Optimizer/rmsprop.h5',custom_objects={'f1_m':f1_m})
rmsprop_complex.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 15, 128)           895360    
                                                                 
 bidirectional_8 (Bidirectio  (None, 15, 256)          263168    
 nal)                                                            
                                                                 
 bidirectional_9 (Bidirectio  (None, 15, 256)          394240    
 nal)                                                            
                                                                 
 bidirectional_10 (Bidirecti  (None, 15, 256)          394240    
 onal)                                                           
                                                                 
 flatten_4 (Flatten)         (None, 3840)              0         
                                                      

In [32]:
rmsprop_complex.evaluate(x_test,y_test)

17/17 [==============================] - 2s 54ms/step - loss: 0.3635 - accuracy: 0.8848 - f1_m: 0.8888


[0.3634621798992157, 0.8848368525505066, 0.8887892365455627]